In [1]:
import pandas as pd
import psycopg2
from psycopg2.extras import RealDictCursor,execute_values
import numpy as np
from sqlalchemy import create_engine
import os
from dotenv import load_dotenv

# Load environment variables (optional, for security)
load_dotenv()



True

In [2]:


DB_CONFIG = {
     'host': os.getenv('DB_HOST'),
     'database': os.getenv('DB_NAME'),
     'user': os.getenv('DB_USER'),
     'password': os.getenv('DB_PASSWORD'),
     'port': os.getenv('DB_PORT'),
     }




In [3]:
def connect_to_database():
    """Create database connection"""
    try:
        connection = psycopg2.connect(**DB_CONFIG)
        print("✅ Successfully connected to PostgreSQL database")
        return connection
    except Exception as e:
        print(f"❌ Error connecting to database: {e}")
        return None

In [4]:

# === 1. Read CSV ===
df = pd.read_csv("./coches_procesado.csv", sep=";", decimal=",", na_values=["", " ", "Sin clasificación"])

# Clean column names (remove leading/trailing spaces)
df.columns = df.columns.str.strip()
print("📌 CSV Columns Detected:", df.columns.tolist())

# === 2. Rename columns to match DB ===
df.rename(columns={
    "Matricula": "matricula",
    "Modelo": "Modelo",
    "Clasificación Energética": "etiqueta",
    "Consumo MIN": "consumo_min",
    "Consumo MAX": "consumo_max",
    "Emisiones MIN (gCO2/km)": "emisiones_min",
    "Emisiones MAX (gCO2/km)": "emisiones_max",
    "Motorización": "motorizacion"
}, inplace=True)

# === 3. Split brand and model ===
df["marca"] = df["Modelo"].apply(lambda x: str(x).split()[0] if pd.notnull(x) else None)
df["modelo"] = df["Modelo"].apply(lambda x: " ".join(str(x).split()[1:]) if pd.notnull(x) and len(str(x).split()) > 1 else None)

# === 4. Add missing DB columns ===
df["tipo"] = df["motorizacion"].apply(
    lambda x: "eléctrico" if "Eléctrico" in str(x) or "eléctrico" in str(x).lower() else None
)
df["km"] = None

# === 5. Reorder to match DB schema (without id_usuario) ===
columns = ["matricula", "marca", "modelo", "etiqueta", "tipo",
           "consumo_min", "consumo_max", "emisiones_min", "emisiones_max",
           "motorizacion", "km"]

records = df[columns].where(pd.notnull(df), None).values.tolist()  # Replace NaN → None

# === 6. Insert into PostgreSQL ===
connection = connect_to_database()
if connection:
    with connection.cursor() as cur:
        insert_query = """
            INSERT INTO vehiculo (
                matricula, marca, modelo, etiqueta, tipo, 
                consumo_min, consumo_max, emisiones_min, emisiones_max, 
                motorizacion, km
            ) VALUES (
                %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s
            )
            ON CONFLICT (matricula) DO NOTHING;
        """

        for row in records:
            cur.execute(insert_query, row)

    connection.commit()
    print(f"✅ Inserted {len(records)} rows into vehiculo")
    connection.close()


FileNotFoundError: [Errno 2] No such file or directory: './coches_procesado.csv'